# Predicting House Prices: Data Analysis and Data Cleaning

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pandas.core.common import random_state
from scipy import stats

## 1. Loading up The Data and Displaying it:

* Let's load the data and analyze its rows columns.


In [5]:
df=pd.read_csv('home_price.csv')
df=df.drop_duplicates() # Dropping duplicated rows
df.info() # Display general information about column names, row count, and data type
print(df.describe()) # Display general information about the data

<class 'pandas.core.frame.DataFrame'>
Index: 20131 entries, 0 to 20325
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Net_Metrekare       20131 non-null  int64  
 1   Brüt_Metrekare      20131 non-null  float64
 2   Oda_Sayısı          19876 non-null  float64
 3   Bulunduğu_Kat       17859 non-null  object 
 4   Eşya_Durumu         12808 non-null  object 
 5   Binanın_Yaşı        20131 non-null  object 
 6   Isıtma_Tipi         20131 non-null  object 
 7   Fiyat               20131 non-null  float64
 8   Şehir               20131 non-null  object 
 9   Binanın_Kat_Sayısı  20131 non-null  int64  
 10  Kullanım_Durumu     20131 non-null  object 
 11  Yatırıma_Uygunluk   14599 non-null  object 
 12  Takas               14289 non-null  object 
 13  Tapu_Durumu         9659 non-null   object 
 14  Banyo_Sayısı        20085 non-null  float64
dtypes: float64(4), int64(2), object(9)
memory usage: 2.5+ MB
 

## 2. Data Analysis:

###   * Empty data:


* from our info method, we can see that several features have missing rows: Tapu_Durumu,Takas, Yatırıma_Uygunluk,Banyo_Sayısı,Oda_Sayısı,Bulunduğu_Kat,Eşya_Durumu.
* for Tapu_Durumu,Takas, Yatırıma_Uygunluk,Oda_Sayısı,Bulunduğu_Kat,Eşya_Durumu, we will be filling the